In [3]:
# !pip install -q git+https://github.com/keras-team/keras-cv.git --upgrade

In [4]:
import tensorflow as tf
import tifffile as tiff
import matplotlib.pyplot as plt
import os
import numpy as np
import pandas as pd
import random
import keras_cv

from tensorflow.keras.metrics import MeanIoU
from sklearn.model_selection import train_test_split
# import segmentation_models as sm
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.losses import BinaryCrossentropy
# from U_net import multi_unet_model
from PIL import Image
# from transformers import MobileViTFeatureExtractor, MobileViTForSemanticSegmentation
import requests


In [5]:
  # from google.colab import drive
  # drive.mount('/content/drive')

In [6]:
DATA_PATH = os.path.join(os.getcwd(), 'data')
# DATA_PATH = "/content/drive/MyDrive/data"
IMG_DIR_PATH = os.path.join(DATA_PATH, 'images')
LABELS_DIR_PATH = os.path.join(DATA_PATH, 'labels')
IMG_DIR_PATH

'd:\\Cellula Tech CV intern\\Week 3 (Satellite Data & Segmentation)\\data\\images'

In [7]:
img_file_paths = [ os.path.join(IMG_DIR_PATH, i) for i in os.listdir(IMG_DIR_PATH)]
img_file_paths.sort()
img_file_paths

['d:\\Cellula Tech CV intern\\Week 3 (Satellite Data & Segmentation)\\data\\images\\0.tif',
 'd:\\Cellula Tech CV intern\\Week 3 (Satellite Data & Segmentation)\\data\\images\\1.tif',
 'd:\\Cellula Tech CV intern\\Week 3 (Satellite Data & Segmentation)\\data\\images\\10.tif',
 'd:\\Cellula Tech CV intern\\Week 3 (Satellite Data & Segmentation)\\data\\images\\100.tif',
 'd:\\Cellula Tech CV intern\\Week 3 (Satellite Data & Segmentation)\\data\\images\\101.tif',
 'd:\\Cellula Tech CV intern\\Week 3 (Satellite Data & Segmentation)\\data\\images\\102.tif',
 'd:\\Cellula Tech CV intern\\Week 3 (Satellite Data & Segmentation)\\data\\images\\103.tif',
 'd:\\Cellula Tech CV intern\\Week 3 (Satellite Data & Segmentation)\\data\\images\\104.tif',
 'd:\\Cellula Tech CV intern\\Week 3 (Satellite Data & Segmentation)\\data\\images\\105.tif',
 'd:\\Cellula Tech CV intern\\Week 3 (Satellite Data & Segmentation)\\data\\images\\106.tif',
 'd:\\Cellula Tech CV intern\\Week 3 (Satellite Data & Segmentati

In [8]:
def normalize_image(image):
    """
    Normalize image to [0, 1] range.
    """
    min_val = np.min(image)
    max_val = np.max(image)
    normalized_image = (image - min_val) / (max_val - min_val)

    return normalized_image

In [9]:
images_arr = []

for i in img_file_paths:
    img = tiff.imread(i)
    images_arr.append(normalize_image(img))

images_arr = np.array(images_arr)

In [10]:
file_names_updated = [i for i in  os.listdir(LABELS_DIR_PATH) if '_' not in i]
file_names_updated.sort()
file_names_updated

['0.png',
 '1.png',
 '10.png',
 '100.png',
 '101.png',
 '102.png',
 '103.png',
 '104.png',
 '105.png',
 '106.png',
 '107.png',
 '108.png',
 '109.png',
 '11.png',
 '110.png',
 '111.png',
 '112.png',
 '113.png',
 '114.png',
 '115.png',
 '116.png',
 '117.png',
 '118.png',
 '119.png',
 '12.png',
 '120.png',
 '121.png',
 '122.png',
 '123.png',
 '124.png',
 '125.png',
 '126.png',
 '127.png',
 '128.png',
 '129.png',
 '13.png',
 '130.png',
 '131.png',
 '132.png',
 '133.png',
 '134.png',
 '135.png',
 '136.png',
 '137.png',
 '138.png',
 '139.png',
 '14.png',
 '140.png',
 '141.png',
 '142.png',
 '143.png',
 '144.png',
 '145.png',
 '146.png',
 '147.png',
 '148.png',
 '149.png',
 '15.png',
 '150.png',
 '151.png',
 '152.png',
 '153.png',
 '154.png',
 '155.png',
 '156.png',
 '157.png',
 '158.png',
 '159.png',
 '16.png',
 '160.png',
 '161.png',
 '162.png',
 '163.png',
 '164.png',
 '165.png',
 '166.png',
 '167.png',
 '168.png',
 '169.png',
 '17.png',
 '170.png',
 '171.png',
 '172.png',
 '173.png',
 '17

In [11]:
labels_arr = []

for i in file_names_updated:
    label_file_path = os.path.join(LABELS_DIR_PATH, i)
    label = plt.imread(label_file_path)
    labels_arr.append(label)

labels_arr = np.array(labels_arr)

In [12]:
label_binary = labels_arr.copy()
label_binary[label_binary > 0] = 1
label_binary[0]

array([[0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(images_arr, label_binary, test_size = 0.10, random_state = 42)

In [14]:
X_train.shape, y_train.shape

((275, 128, 128, 12), (275, 128, 128))

In [15]:
X_test.shape, y_test.shape

((31, 128, 128, 12), (31, 128, 128))

In [16]:
IMG_HEIGHT = X_train.shape[1]
IMG_WIDTH  = X_train.shape[2]
IMG_CHANNELS = X_train.shape[3]

In [17]:
IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS

(128, 128, 12)

In [18]:
# import tensorflow as tf
# from tensorflow.keras import layers, models
# import tensorflow_hub as hub

# # Load the DeepLabV3 feature extractor from TensorFlow Hub
# base_model = hub.KerasLayer("https://tfhub.dev/tensorflow/deeplabv3/1", trainable=False)

# # Define a custom model
# def build_custom_deeplabv3(input_shape=(128, 128, 12), num_classes=2):
#     inputs = layers.Input(shape=input_shape)

#     # Pass through the pre-trained DeepLabV3 base (remove output layers)
#     x = base_model(inputs)

#     # Add a custom classifier layer for 2 classes (binary or 2-class segmentation)
#     x = layers.Conv2D(num_classes, (1, 1), padding='same', activation='softmax')(x)

#     # Create the final model
#     model = models.Model(inputs=inputs, outputs=x)

#     return model

# # Build the model with your custom input shape and 2 output classes
# model = build_custom_deeplabv3(input_shape=(128, 128, 12), num_classes=2)

# # Print the model summary
# model.summary()


In [19]:
# import tensorflow as tf
# import tensorflow_hub as hub
# import numpy as np
# import PIL.Image as Image
# import matplotlib.pyplot as plt

# # Load the DeepLabV3 model from TensorFlow Hub
# model_url = "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1"  # Replace with DeepLabV3 URL
# model = hub.load(model_url)

# # Load and preprocess an image
# def preprocess_image(image_path):
#     img = Image.open(image_path).resize((513, 513))  # DeepLabV3 typically uses 513x513
#     img = np.array(img) / 255.0  # Normalize pixel values
#     return img

# image_path = "path_to_your_image.jpg"  # Provide the path to your image
# image = preprocess_image(image_path)

# # Add batch dimension
# input_image = tf.expand_dims(image, axis=0)

# # Run inference
# output = model(input_image)
# predicted_mask = tf.argmax(output['semantic'], axis=-1)
# predicted_mask = predicted_mask[0]  # Remove batch dimension

# # Visualize the segmentation
# plt.figure(figsize=(10, 10))
# plt.imshow(predicted_mask)
# plt.axis('off')
# plt.show()


In [20]:
# Create IoU metric
iou_metric = MeanIoU(2)
# model.compile(optimizer='adam', loss=BinaryCrossentropy(), metrics=[iou_metric, 'accuracy'])


In [38]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
import keras_cv

# Define the custom input shape
input_shape = (128, 128, 12)

# Use a compatible backbone from keras_cv
backbone = keras_cv.models.ResNet50V2Backbone.from_preset("resnet50_v2_imagenet")

# Create a preprocessing model to adjust the number of channels
def create_preprocessing_model(input_shape):
    inputs = Input(shape=input_shape)
    x = Conv2D(3, (1, 1), padding='same')(inputs)
    return Model(inputs=inputs, outputs=x)

preprocessing_model = create_preprocessing_model(input_shape)

# Initialize the DeepLabV3+ model
model = keras_cv.models.segmentation.DeepLabV3Plus(
    num_classes=1,
    backbone=backbone
)
for layer in model.layers:
    layer.trainable = False
    
for layer in model.layers[-4:]:
    layer.trainable = True

# for layer in model.layers[:2]:
#     layer.trainable = True
# Connect the preprocessing model to the DeepLabV3+ model
inputs = Input(shape=input_shape)
processed_inputs = preprocessing_model(inputs)
outputs = model(processed_inputs)
final_model = Model(inputs=inputs, outputs=outputs)

# Print model summary
print(final_model.summary())

Model: "functional_86"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_86 (InputLayer)     │ (None, 128, 128, 12)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ functional_76 (Functional)      │ (None, 128, 128, 3)    │            39 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ deep_lab_v3_plus_7              │ (None, 128, 128, 1)    │    39,186,368 │
│ (DeepLabV3Plus)                 │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 39,186,407 (149.48 MB)

 Trainable params: 91,015 (355.53 KB)

 Non-trainable params: 39,095,392 (149.14 MB)

None


In [39]:
final_model.compile(optimizer='adam', loss=BinaryCrossentropy(), metrics=[iou_metric, 'accuracy'])

In [40]:
final_model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 77s 8s/step - accuracy: 0.2628 - loss: 11.7528 - mean_io_u: 0.1314 - val_accuracy: 0.1814 - val_loss: 13.0506 - val_mean_io_u: 0.0907
Epoch 2/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 73s 8s/step - accuracy: 0.2502 - loss: 11.9530 - mean_io_u: 0.1251 - val_accuracy: 0.1814 - val_loss: 13.0506 - val_mean_io_u: 0.0907
Epoch 3/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 68s 8s/step - accuracy: 0.2359 - loss: 12.1819 - mean_io_u: 0.1179 - val_accuracy: 0.1814 - val_loss: 13.0506 - val_mean_io_u: 0.0907
Epoch 4/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 76s 8s/step - accuracy: 0.2752 - loss: 11.5550 - mean_io_u: 0.1376 - val_accuracy: 0.1814 - val_loss: 13.0506 - val_mean_io_u: 0.0907
Epoch 5/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 71s 8s/step - accuracy: 0.2685 - loss: 11.6615 - mean_io_u: 0.1343 - val_accuracy: 0.1814 - val_loss: 13.0506 - val_mean_io_u: 0.0907
Epoch 6/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 64s 7s/step - accuracy: 0.2732 - loss: 11.5871 - mean_io_u: 0.1366 - val_accuracy: 0.1814 - val_loss: 13.0506 - va

In [41]:
history = final_model.fit(X_train, y_train, batch_size=70, epochs=10, validation_data=(X_test, y_test))


Epoch 1/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 88s 22s/step - accuracy: 0.2629 - loss: 11.7510 - mean_io_u: 0.1315 - val_accuracy: 0.1814 - val_loss: 13.0506 - val_mean_io_u: 0.0907
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 76s 18s/step - accuracy: 0.2844 - loss: 11.4081 - mean_io_u: 0.1422 - val_accuracy: 0.1814 - val_loss: 13.0506 - val_mean_io_u: 0.0907
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 74s 18s/step - accuracy: 0.2611 - loss: 11.7797 - mean_io_u: 0.1306 - val_accuracy: 0.1814 - val_loss: 13.0506 - val_mean_io_u: 0.0907
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 74s 19s/step - accuracy: 0.2643 - loss: 11.7296 - mean_io_u: 0.1321 - val_accuracy: 0.1814 - val_loss: 13.0506 - val_mean_io_u: 0.0907
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 81s 21s/step - accuracy: 0.2555 - loss: 11.8685 - mean_io_u: 0.1278 - val_accuracy: 0.1814 - val_loss: 13.0506 - val_mean_io_u: 0.0907
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 78s 20s/step - accuracy: 0.2715 - loss: 11.6147 - mean_io_u: 0.1357 - val_accuracy: 0.1814 - val_loss: 13.050